In [1]:
import os

import numpy as np
import pandas as pd
import scanpy as sc
from sklearn.neural_network import MLPClassifier

import matplotlib.pyplot as plt
import seaborn as sns

sc.settings.n_jobs = 24
sc.settings.set_figure_params(dpi=180, dpi_save=300, frameon=False, figsize=(4, 4), fontsize=8, facecolor='white')

In [2]:
import numpy as np
import pandas as pd
import anndata as ad
from scipy.sparse import csr_matrix
print(ad.__version__)

0.8.0


In [3]:
%run "C:\Users\shiwei\Documents\ImageAnalysis3\required_files\Startup_py3.py"
sys.path.append(r"C:\Users\shiwei\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

import h5py
from ImageAnalysis3.classes import _allowed_kwds
import ast

import pandas as pd

50760


In [4]:
# Chromatin_analysis_tools (ATC)
# Get path for the py containing functions
import os
import sys
import importlib
module_path =r'C:\Users\shiwei\Documents\AnalysisTool_Chromatin'
if module_path not in sys.path:
    sys.path.append(module_path)
    
# import relevant modules
import gene_selection 
importlib.reload(gene_selection)
import gene_to_loci
importlib.reload(gene_to_loci)
import gene_activity
importlib.reload(gene_activity)
import loci_1d_features
importlib.reload(loci_1d_features)  

import atac_to_loci
importlib.reload(atac_to_loci)

<module 'atac_to_loci' from 'C:\\Users\\shiwei\\Documents\\AnalysisTool_Chromatin\\atac_to_loci.py'>

In [5]:
%matplotlib inline

sc.settings.set_figure_params(dpi=300, )

# 1. Load atac Data

In [6]:
# Get loaded adata from other notebook
import os
import scanpy as sc
# L drive is Crick Pu_SSD_0
scRNA_folder =r'L:\Shiwei\DNA_MERFISH_analysis\Paired_tag\anndata'
# load from here for saved h5ad
target_mode = 'H3K27me3'
adata = sc.read(os.path.join(scRNA_folder,f'FC_pairtag_{target_mode}.h5ad'))

In [7]:
adata_ori = adata
adata_ori.var

""
chr10_10000000_10005000
chr10_100000000_100005000
chr10_100005000_100010000
chr10_100010000_100015000
chr10_100015000_100020000
...
chrY_9945000_9950000
chrY_995000_1000000
chrY_9975000_9980000
chrY_9985000_9990000


# 2. Load MERFISH codebook

In [8]:
# load codebook
# L drive is Crick Pu_SSD_0
analysis_save_folder=r'\\10.245.74.158\Chromatin_NAS_8\Analyzed_data\MouseBrain_PostAnalysis_20230201'

# Load sorted codebook (allows direct matrix slicing) with cell type info
# Load codebook (then sort to allow direct matrix slicing) with cell type info

celltype_codebook_fname = os.path.join(analysis_save_folder,'merged_codebook.csv')
celltype_codebook_df = pd.read_csv (celltype_codebook_fname, index_col=0)

# sort df temporailiy so matrix can be sliced by df order directly
celltype_codebook_df = loci_1d_features.sort_loci_df_by_chr_order (celltype_codebook_df)

celltype_codebook_df[['name','chr','chr_order']].head()

,name,chr,chr_order
0,1:3742742-3759944,1,0.0
1,1:6245958-6258969,1,1.0
2,1:8740008-8759916,1,2.0
1016,1:9627926-9637875,1,3.0
1017,1:9799472-9811359,1,4.0


In [9]:
# keep relevant info and reset the format
import gene_to_loci as gl

celltype_codebook_df['loci_name']= celltype_codebook_df['name'].map(gl.loci_pos_format) 
celltype_codebook_df['loci_name']= celltype_codebook_df['loci_name'].map(lambda x: x[0])
celltype_codebook_df['chr_order']= celltype_codebook_df['chr_order'].map(lambda x: int(x))
celltype_codebook_df = celltype_codebook_df.set_index('loci_name')
celltype_codebook_df[['chr','chr_order']]

,chr,chr_order
loci_name,,
chr1_3742742_3759944,1,0
chr1_6245958_6258969,1,1
chr1_8740008_8759916,1,2
chr1_9627926_9637875,1,3
chr1_9799472_9811359,1,4
...,...,...
chrX_166247682_166259932,X,60
chrX_167157164_167167452,X,61
chrX_168746045_168757590,X,62


# 3. Load peaks for all MERFISH loci

In [10]:
import atac_to_loci
importlib.reload(atac_to_loci)

<module 'atac_to_loci' from 'C:\\Users\\shiwei\\Documents\\AnalysisTool_Chromatin\\atac_to_loci.py'>

In [13]:
peak_names_list = adata_ori.var.index.tolist()


loci_peak_df = celltype_codebook_df[['name','id','chr','chr_order','library']]


# prepare and save this for future analyses
#for extend_dist in [0*1000,50*1000,100*1000,500*1000,1000*1000,2000*1000]:
for extend_dist in [2000*1000]:  
                    
    loci_peak_df = atac_to_loci.find_peaks_near_gene_dataframe (loci_peak_df, 
                                        peak_names_list, 
                                        sel_loci_col = None,
                                        extend_dist = extend_dist,
                                        peak_coverage_type = 'center',
                                        key_added = None)

100%|████████████████████████████████████████████████████████████████████████████| 1982/1982 [1:18:44<00:00,  2.38s/it]


In [15]:
output_folder = scRNA_folder

loci_peak_df.to_csv(os.path.join(output_folder, f'MERFISH_loci_adjacent_{target_mode}_center.csv'))

In [11]:
peak_names_list = adata_ori.var.index.tolist()


loci_peak_df = celltype_codebook_df[['name','id','chr','chr_order','library']]


# prepare and save this for future analyses
#for extend_dist in [0*1000,50*1000,100*1000,500*1000,1000*1000,2000*1000]:
for extend_dist in [0*1000,50*1000,500*1000,]:  
                    
    loci_peak_df = atac_to_loci.find_peaks_near_gene_dataframe (loci_peak_df, 
                                        peak_names_list, 
                                        sel_loci_col = None,
                                        extend_dist = extend_dist,
                                        peak_coverage_type = 'center',
                                        key_added = None)

100%|████████████████████████████████████████████████████████████████████████████| 1982/1982 [1:14:50<00:00,  2.27s/it]


In [12]:
loci_peak_df

,name,id,chr,chr_order,library,adjacent_peaks_0kb_center,adjacent_peaks_50kb_center,adjacent_peaks_500kb_center
loci_name,,,,,,,,
chr1_3742742_3759944,1:3742742-3759944,1,1,0,CTP11,chr1_3745000_3750000; chr1_3750000_3755000; ch...,chr1_3695000_3700000; chr1_3700000_3705000; ch...,chr1_3245000_3250000; chr1_3250000_3255000; ch...
chr1_6245958_6258969,1:6245958-6258969,2,1,1,CTP11,chr1_6245000_6250000; chr1_6250000_6255000; ch...,chr1_6195000_6200000; chr1_6200000_6205000; ch...,chr1_5745000_5750000; chr1_5755000_5760000; ch...
chr1_8740008_8759916,1:8740008-8759916,3,1,2,CTP11,chr1_8740000_8745000; chr1_8745000_8750000; ch...,chr1_8695000_8700000; chr1_8700000_8705000; ch...,chr1_8240000_8245000; chr1_8255000_8260000; ch...
chr1_9627926_9637875,1:9627926-9637875,1,1,3,CTP13,chr1_9630000_9635000; chr1_9635000_9640000,chr1_9580000_9585000; chr1_9590000_9595000; ch...,chr1_10000000_10005000; chr1_10005000_10010000...
chr1_9799472_9811359,1:9799472-9811359,2,1,4,CTP13,chr1_9800000_9805000; chr1_9805000_9810000,chr1_9750000_9755000; chr1_9755000_9760000; ch...,chr1_10000000_10005000; chr1_10005000_10010000...
...,...,...,...,...,...,...,...,...
chrX_166247682_166259932,X:166247682-166259932,1059,X,60,CTP11,chrX_166250000_166255000; chrX_166255000_16626...,chrX_166200000_166205000; chrX_166205000_16621...,chrX_165750000_165755000; chrX_165755000_16576...
chrX_167157164_167167452,X:167157164-167167452,990,X,61,CTP13,chrX_167160000_167165000,chrX_167105000_167110000; chrX_167115000_16712...,chrX_166655000_166660000; chrX_166660000_16666...
chrX_168746045_168757590,X:168746045-168757590,1060,X,62,CTP11,chrX_168750000_168755000; chrX_168755000_16876...,chrX_168695000_168700000; chrX_168705000_16871...,chrX_168245000_168250000; chrX_168250000_16825...


In [13]:
output_folder = scRNA_folder

loci_peak_df.to_csv(os.path.join(output_folder, f'MERFISH_loci_adjacent_{target_mode}_center_other_res.csv'))